In [30]:
pip install six


Note: you may need to restart the kernel to use updated packages.


In [31]:
import pandas as pd
import sqlite3
from airflow import DAG
from airflow.operators.python_operator import PythonOperator
from datetime import datetime, timedelta
import os
from kafka import KafkaConsumer
import pandas as pd

/var/folders/ty/kkyy7c9s4059czrtyxg_0mz40000gn/T/ipykernel_10683/499173985.py:4 DeprecationWarning: The `airflow.operators.python_operator.PythonOperator` class is deprecated. Please use `'airflow.operators.python.PythonOperator'`.

ModuleNotFoundError: No module named 'kafka.vendor.six.moves'

In [7]:
df = pd.read_csv('/Users/jacksonsorenson/Documents/Ecommerce Data Project/Original CSV/data.csv', 
                 encoding='latin1')


In [8]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [9]:
df.describe

<bound method NDFrame.describe of        InvoiceNo StockCode                          Description  Quantity  \
0         536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1         536365     71053                  WHITE METAL LANTERN         6   
2         536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3         536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4         536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   
...          ...       ...                                  ...       ...   
541904    581587     22613          PACK OF 20 SPACEBOY NAPKINS        12   
541905    581587     22899         CHILDREN'S APRON DOLLY GIRL          6   
541906    581587     23254        CHILDRENS CUTLERY DOLLY GIRL          4   
541907    581587     23255      CHILDRENS CUTLERY CIRCUS PARADE         4   
541908    581587     22138        BAKING SET 9 PIECE RETROSPOT          3   

            InvoiceDate  UnitPrice  Custo

In [12]:
df.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [14]:
df = df.dropna(subset = ['Description'])
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [19]:
def store_data(df, db_path='ecommerce_pipeline.db'):
    conn = sqlite3.connect(db_path)
    df.to_sql('transactions', conn, if_exists='append', index=False)
    
    print("Data Stored Successfully!")
    conn.close()

store_data(df)

Data Stored Successfully!


In [26]:
default_args = {
    'start_date': datetime(2024, 6, 1),
    'retries': 1,
    'retry_delay': timedelta(minutes=5),
}

dag = DAG('ecommerce_data_pipeline', default_args=default_args, schedule_interval='@daily')

def load_data_task():
    os.system('python load_data.py')

def clean_data_task():
    os.system('python clean_data.py')

def store_data_task():
    os.system('python store_data.py')

load_data = PythonOperator(task_id='load_data', python_callable=load_data_task, dag=dag)
clean_data = PythonOperator(task_id='clean_data', python_callable=clean_data_task, dag=dag)
store_data = PythonOperator(task_id='store_data', python_callable=store_data_task, dag=dag)

load_data >> clean_data >> store_data

/var/folders/ty/kkyy7c9s4059czrtyxg_0mz40000gn/T/ipykernel_10683/3492318530.py:7 RemovedInAirflow3Warning: Param `schedule_interval` is deprecated and will be removed in a future release. Please use `schedule` instead.

<Task(PythonOperator): store_data>